In [3]:
import os
import re
import sys
import torch
import hashlib
import itertools
import logging

logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')


from progress.bar import Bar
from concurrent.futures import ProcessPoolExecutor

from Utils.NotesSeq import NoteSeq as ns
from Utils.EventSeq import EventSeq as es
from Utils.ControlSeq import ControlSeq as cs
from Utils import utils

import warnings
warnings.filterwarnings("ignore")

In [4]:
def preprocess_midi(path):

    note_seq = ns.from_midi_file(path)
    iter = itertools.cycle(note_seq)

    es1 = list
    es2 = list

    cs1 = list
    cs2 = list
    ev = True
    #print("evaluating")
    for seq in iter:
        #print(seq)
        if ev:
            #print("1st iter")
            seq.adjust_time(-seq.notes[0].start)
            event_seq = es.from_note_seq(seq)
            control_seq = cs.from_event_seq(event_seq)
            es1 = event_seq.to_array()
            cs1 = control_seq.to_compressed_array()
            ev = False
        else:
            #print("2nd iter")
            seq.adjust_time(-seq.notes[0].start)
            event_seq = es.from_note_seq(seq)
            control_seq = cs.from_event_seq(event_seq)
            es2 = event_seq.to_array()
            cs2 = control_seq.to_compressed_array()
            break

    return es1, es2, cs1, cs2

In [5]:
midi_path = './Naruto_Shippuden_-_Silhouette.mid'

In [ ]:
def from_midi_file(path):
        midi = PrettyMIDI(path)
        track_len = len(midi.instruments)

        if(track_len < 3):
            stream1 = NoteSeq.from_tracks(midi,[0])
            stream2 = NoteSeq.from_tracks(midi,[1])
            #print(" 2 tracce : " + path)
        elif(track_len < 4):
            stream1 = NoteSeq.from_tracks(midi,[0])
            stream2 = NoteSeq.from_tracks(midi,[1,2])
            #print(" 3 tracce : " + path)
        else:
            stream1 = NoteSeq.from_tracks(midi,[0,1])
            stream2 = NoteSeq.from_tracks(midi,[2,3])
            #print(" 4 tracce : " + path)

        return stream1,stream2